In [14]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.vectorstores import FAISS
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

In [35]:
embeddings = OllamaEmbeddings(model="llama3")
llm = ChatOllama(model="llama3")
#llm = ChatGroq(model="qwen-qwq-32b")
llm.invoke("capital of india?")

AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={'model': 'llama3', 'created_at': '2025-04-26T16:26:11.536786Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1248233750, 'load_duration': 818509709, 'prompt_eval_count': 14, 'prompt_eval_duration': 252000000, 'eval_count': 9, 'eval_duration': 176000000, 'model_name': 'llama3'}, id='run-a9bb829f-c593-4c94-bced-7a14a140156d-0', usage_metadata={'input_tokens': 14, 'output_tokens': 9, 'total_tokens': 23})

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=200)
api_wrapper


WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/m0k08nh/projects/preparing/test/venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=5, lang='en', load_all_available_meta=False, doc_content_chars_max=200)

In [49]:
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [23]:
from langchain_community.document_loaders import WebBaseLoader
urls = ["https://langchain-ai.github.io/langgraph/"]
loader = WebBaseLoader(urls)
docs = loader.load()
docs

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/', 'title': 'LangGraph', 'description': 'Build reliable, stateful AI systems, without giving up control', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\nJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            LangGraph\n          \n\n\n\n            \n              LangGraph\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n\n\n          \n  \n    \n  \n  LangGraph\n\n        \n\n\n\n          \n  \n    \n  \n  Agents\n\n        \n\n\n\n          \n  \n    \n  \n  API reference\n\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n    LangGraph\n  \n\n\n\n\n\n\n    GitHub\n  \n\n\

In [32]:
documents = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100).split_documents(docs)
documents[2]
len(documents)

21

In [36]:
vector_db = FAISS.from_documents(documents=documents,embedding=embeddings)

In [37]:
vector_db.similarity_search("what is langgraph")

[Document(id='aec37873-1843-48d0-8043-5a55c2d8347c', metadata={'source': 'https://langchain-ai.github.io/langgraph/', 'title': 'LangGraph', 'description': 'Build reliable, stateful AI systems, without giving up control', 'language': 'en'}, page_content='Learn the basics\n  \n\n    \n  \n\n\n\n\n\n    \n  \n    Deployment\n  \n\n    \n  \n\n\n\n\n\n\n\n\n\n\n    \n  \n    Guides\n  \n\n    \n  \n\n\n\n\n\n            \n  \n    Guides\n  \n\n          \n\n\n\n\n    \n  \n    How-to Guides\n  \n\n    \n  \n\n\n\n\n\n\n    \n  \n    Concepts\n  \n\n    \n  \n\n\n\n\n\n\n    \n  \n    Tutorials\n  \n\n    \n  \n\n\n\n\n\n\n\n\n\n\n    \n  \n    Resources\n  \n\n    \n  \n\n\n\n\n\n            \n  \n    Resources\n  \n\n          \n\n\n\n\n    \n  \n    Companies using LangGraph\n  \n\n    \n  \n\n\n\n\n\n    \n  \n    LLMS-txt'),
 Document(id='28f3fcc1-f89f-4af7-85f7-17eb539c2fbe', metadata={'source': 'https://langchain-ai.github.io/langgraph/', 'title': 'LangGraph', 'description': 'Build r

In [39]:
retriever = vector_db.as_retriever()

In [51]:
from langchain.tools.retriever import create_retriever_tool
langgraph_tool = create_retriever_tool(retriever=retriever,name="langgraph_search",description="search about langgraph")

In [52]:
langgraph_tool.name

'langgraph_search'

In [48]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [53]:
tools = [wiki,arxiv,langgraph_tool]

In [54]:
## Agents
from langchain.agents import create_openai_tools_agent

In [55]:
from langchain import hub

In [56]:
llm_with_tools = llm.bind_tools(tools=tools)